## Trump Model Training

In [122]:
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.utils import to_categorical
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf


In [123]:
## define paths to data files
path_to_train_data = Path("C:\\Users\\matth\\Documents\\DL4G\\jass-data\\split\\train\\filtered\\card\\NEW\\csv")
path_to_test_data = Path("C:\\Users\\matth\\Documents\\DL4G\\jass-data\\split\\test\\filtered\\card\\NEW\\csv")
path_to_val_data = Path("C:\\Users\\matth\\Documents\\DL4G\\jass-data\\split\\val\\filtered\\card\\NEW\\csv")

In [124]:
# train data
data_train1 = pd.read_csv(path_to_train_data / '0001.csv', header=None)
data_train2 = pd.read_csv(path_to_train_data / '0002.csv', header=None)
data_train3 = pd.read_csv(path_to_train_data / '0003.csv', header=None)
data_train4 = pd.read_csv(path_to_train_data / '0004.csv', header=None)
data_train5 = pd.read_csv(path_to_train_data / '0005.csv', header=None)
data_train6 = pd.read_csv(path_to_train_data / '0006.csv', header=None)
data_train7 = pd.read_csv(path_to_train_data / '0007.csv', header=None)
data_train8 = pd.read_csv(path_to_train_data / '0008.csv', header=None)
data_train9 = pd.read_csv(path_to_train_data / '0009.csv', header=None)


data_val1 = pd.read_csv(path_to_val_data / '0001.csv', header=None)
data_val2 = pd.read_csv(path_to_val_data / '0002.csv', header=None)
data_val3 = pd.read_csv(path_to_val_data / '0003.csv', header=None)

data_train = pd.concat([data_train1, data_train2, data_train3, data_train4, data_train5, data_train6, data_train7, data_train8, data_train9, data_val1, data_val2, data_val3], axis=0)
data_train.head()
#data_val = pd.concat([data_val1, data_val2], axis=0)

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,0,1,1,0,0,1,1,0,0,...,1,0,0,0,0,0,1,0,0,22
1,1,0,0,0,1,1,0,1,1,0,...,0,0,1,0,0,0,0,1,0,32
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,28
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,21
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,35


In [125]:
# test data
data_test1 = pd.read_csv(path_to_test_data / '0001.csv', header=None)
data_test2 = pd.read_csv(path_to_train_data / '0002.csv', header=None)
data_test3 = pd.read_csv(path_to_train_data / '0003.csv', header=None)

data_test = pd.concat([data_test1, data_test2, data_test3])
data_test.shape
data_test.head()

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,1,0,1,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,32
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,25
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,18
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,14
4,0,1,0,1,0,0,0,1,0,1,...,1,0,0,1,0,0,0,0,0,3


## Label data for readability

## Prepare x and y values

In [126]:
# colums used for extracting x and y values. The same effect could be achieved with train_test_split-Method, but 
# since we already have different files, we dont need to split the files using this method.
#data_X_columns = cards + forehand
#data_Y_colums = trump

x_train = data_train[data_train.columns[0:82]]
y_train = data_train[data_train.columns[82]]
#print (x_train.head())
#print (y_train.head())

x_test = data_test[data_test.columns[0:82]]
y_test = data_test[data_test.columns[82]]

## Create model and train it

In [127]:
# We have 37 features, so we have a node for each feature. There are 7 output categories: each trump color(4), 
# obe-abe, unde-ufe, schiebe. So we need an reducing function with 7 elements
model = keras.Sequential()
model.add(keras.layers.Dense(82, activation='relu', input_shape=[82]))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(36, activation='softmax'))

opt = Adam()

model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

print(model.summary())


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_116 (Dense)            (None, 82)                6806      
_________________________________________________________________
dense_117 (Dense)            (None, 100)               8300      
_________________________________________________________________
dense_118 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_119 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_120 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_121 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_122 (Dense)            (None, 100)             

In [ ]:
y_train_categorical = to_categorical(y_train)

callbacks = [EarlyStopping(monitor='val_loss', patience=10),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

history = model.fit(x_train, y_train_categorical, validation_split=0.25, epochs=100, batch_size=128, callbacks=callbacks)

Train on 891054 samples, validate on 297018 samples
Epoch 1/100
891054/891054 [==============================] - 39s 44us/sample - loss: 1.7788 - accuracy: 0.4644 - val_loss: 1.4850 - val_accuracy: 0.5288
Epoch 2/100
891054/891054 [==============================] - 41s 46us/sample - loss: 1.3951 - accuracy: 0.5496 - val_loss: 1.3433 - val_accuracy: 0.5629
Epoch 3/100
891054/891054 [==============================] - 40s 45us/sample - loss: 1.2833 - accuracy: 0.5746 - val_loss: 1.2460 - val_accuracy: 0.5818
Epoch 4/100
891054/891054 [==============================] - 38s 42us/sample - loss: 1.2223 - accuracy: 0.5884 - val_loss: 1.2159 - val_accuracy: 0.5885
Epoch 5/100
891054/891054 [==============================] - 39s 44us/sample - loss: 1.1806 - accuracy: 0.5967 - val_loss: 1.1691 - val_accuracy: 0.5981
Epoch 6/100
891054/891054 [==============================] - 41s 46us/sample - loss: 1.1473 - accuracy: 0.6033 - val_loss: 1.1353 - val_accuracy: 0.6015
Epoch 7/100
891054/891054 [===

## Generate graphs for loss and accuracy

In [ ]:
plt.plot(history.history['loss'])
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['Train', 'Val'], loc='upper left')

## Test model

In [ ]:
y_test_categorical = to_categorical(y_test)
model.evaluate(x_test, y_test_categorical)

In [ ]:
model.save("card_model_more_data.h5")